In [1]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)

docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
USER_AGENT environment variable not set, consider setting it to identify your requests.


Total characters: 43047


In [2]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, # chunk size (characters)
    chunk_overlap = 200, # chunk overlap (characters)
    add_start_index=True, # track index in original document
)

all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 63 sub-documents.


In [4]:
import os
from dotenv import load_dotenv
# 改用通义千问dashboard SDK调用
from langchain_community.embeddings import DashScopeEmbeddings

load_dotenv()

os.environ["DASHSCOPE_API_KEY"] = os.getenv("API_KEY")  # 通义千问密钥

embeddings = DashScopeEmbeddings(
    model="text-embedding-v4"
)

In [5]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['c2d42c91-c38e-4ac3-92c1-6a498f72869a', 'be8ff9cd-10a0-43e2-b743-f999db1f7683', '49eaca5c-c419-4e3d-977c-6cbc599f283b']


In [6]:
from langchain.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}") for doc in retrieved_docs
    )

    return serialized, retrieved_docs

In [7]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent

## 修改timeout为30，前值为10。否则易造成请求超时。
# 试试qwen3-max的结构化输出效果，模型qwen-max的效果不行
model = init_chat_model("openai:qwen3-max",
                        temperature=0.5,
                        timeout=30,
                        max_tokens=1000,
                        base_url=os.getenv('BASE_URL'),
                        api_key=os.getenv('API_KEY'))

tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)

agent = create_agent(model, tools, system_prompt=prompt)

In [8]:
query = (
    "What is the standard method for Task Decomposition?\n\n"
    "Once you get the answer, look up common extensions of that method."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the standard method for Task Decomposition?

Once you get the answer, look up common extensions of that method.
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_5902c87b401d40d2b46310d5)
 Call ID: call_5902c87b401d40d2b46310d5
  Args:
    query: standard method for Task Decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 1638}
Content: Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test

##### 另外一种方式

In [9]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message

agent = create_agent(model, tools=[], middleware=[prompt_with_context])

In [10]:
query = "What is task decomposition?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is task decomposition?
================================== Ai Message ==================================

**Task decomposition** is the process of breaking down a complex or large task into smaller, more manageable subtasks or steps. This technique helps make difficult problems easier to solve by organizing them into a sequence or structure of simpler components that can be addressed individually.

In the context of **AI agents and large language models (LLMs)**, task decomposition is a key part of planning. It enables the model to:

- Understand the structure of a complex request,
- Identify intermediate goals or subgoals,
- Execute or reason about each part step by step.

### Common Approaches to Task Decomposition:

1. **Chain of Thought (CoT)**  
   - Instructs the model to “think step by step.”
   - Transforms a big task into a linear sequence of simpler reasoning steps.
   - Enhances performance